In [3]:
import cv2
import mediapipe as mp
import numpy as np
from tkinter import Tk, filedialog

# Initialize MediaPipe modules
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)

# Load earring images (PNG with transparency)
left_earring = cv2.imread("left_ear.png", cv2.IMREAD_UNCHANGED)
right_earring = cv2.imread("right_ear.png", cv2.IMREAD_UNCHANGED)

# Ensure alpha channel exists and remove white background from earrings
def remove_white_background(image):
    if image is None:
        print("Error: Earring image not found!")
        return None
    
    if image.shape[-1] == 3:  # Convert BGR to BGRA
        image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)

    # Create mask where white pixels are detected
    gray = cv2.cvtColor(image[:, :, :3], cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)  # White removal

    # Apply mask as alpha channel
    image[:, :, 3] = mask
    return image

left_earring = remove_white_background(left_earring)
right_earring = remove_white_background(right_earring)

# Function to overlay images with alpha blending
def overlay_image(background, overlay, x, y, width, height):
    if overlay is None or background is None:
        return background

    overlay = cv2.resize(overlay, (width, height))
    alpha_mask = overlay[:, :, 3] / 255.0
    alpha_inv = 1.0 - alpha_mask

    h, w, _ = background.shape
    x1, x2 = max(0, x), min(w, x + width)
    y1, y2 = max(0, y), min(h, y + height)

    if x1 >= x2 or y1 >= y2:
        return background

    overlay = overlay[: y2 - y1, : x2 - x1]
    alpha_mask = alpha_mask[: y2 - y1, : x2 - x1]
    alpha_inv = alpha_inv[: y2 - y1, : x2 - x1]

    alpha_mask = np.stack([alpha_mask] * 3, axis=-1)
    alpha_inv = np.stack([alpha_inv] * 3, axis=-1)

    background[y1:y2, x1:x2, :3] = (
        alpha_inv * background[y1:y2, x1:x2, :3] + alpha_mask * overlay[:, :, :3]
    ).astype(np.uint8)

    return background

# Open file dialog to select an image
Tk().withdraw()  # Hide the root window
image_path = filedialog.askopenfilename(title="Select an Image", filetypes=[("Images", "*.jpg;*.jpeg;*.png")])

if not image_path:
    print("No image selected. Exiting.")
    exit()

frame = cv2.imread(image_path)

if frame is None:
    print("Error: Image not found!")
    exit()

# Convert image to RGB for face detection
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
face_result = face_mesh.process(frame_rgb)

if face_result.multi_face_landmarks:
    face_landmarks = face_result.multi_face_landmarks[0].landmark

    left_ear = face_landmarks[234]  # Landmark near left ear
    right_ear = face_landmarks[454]  # Landmark near right ear
    chin = face_landmarks[152]  # Chin landmark for scaling earring size

    if left_ear and right_ear:
        x1, y1 = int(left_ear.x * frame.shape[1]), int(left_ear.y * frame.shape[0])
        x2, y2 = int(right_ear.x * frame.shape[1]), int(right_ear.y * frame.shape[0])
        chin_y = int(chin.y * frame.shape[0])

        # Earring size dynamically based on face height
        earring_width = int(abs(x2 - x1) * 0.3)
        earring_height = int(abs(y1 - chin_y) * 0.9)

        # Move earrings downward slightly for a natural hanging effect
        y1 += int(earring_height * 0.3)  # Move left earring down
        y2 += int(earring_height * 0.3)  # Move right earring down

        # Overlay earrings exactly below the ears
        frame = overlay_image(frame, left_earring, x1 - earring_width // 2, y1, earring_width, earring_height)
        frame = overlay_image(frame, right_earring, x2 - earring_width // 2, y2, earring_width, earring_height)

# Resize output image to a smaller size (640px width while maintaining aspect ratio)
output_width = 640
scale_factor = output_width / frame.shape[1]
output_height = int(frame.shape[0] * scale_factor)
frame_resized = cv2.resize(frame, (output_width, output_height))

# Save and display the resized final image
output_path = "output_with_earrings_resized.png"
cv2.imwrite(output_path, frame_resized)
cv2.imshow("Earrings Try-On", frame_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()
